In [1]:
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import statistics as stat
import datetime
import time
import multiprocessing as mp

from matplotlib.font_manager import FontProperties
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.cluster import AgglomerativeClustering
from sklearn import manifold
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from abc import ABCMeta, abstractmethod

%matplotlib inline
plt.rcParams['axes.unicode_minus']=False
engine = create_engine('sqlite:///fund.db')

In [2]:
class ClusterStrategy(metaclass=ABCMeta):
    @abstractmethod
    def Clustering(self,features):
        pass

In [3]:
class ClusterMethod:
    def __init__(self, clusterStrategy):
        self.clusterStrategy = clusterStrategy
    
    def startClustering(self,features):
        return self.clusterStrategy.Clustering(features)

In [4]:
class K_Means(ClusterStrategy):
    def Clustering(self,features):
        return KMeans(n_clusters=4, n_jobs=1).fit(features)

In [5]:
class Hierarchical(ClusterStrategy):
    def Clustering(self,features):
        return AgglomerativeClustering(n_clusters=4).fit(features)

In [6]:
class CountWinRate(metaclass=ABCMeta):
    
    '''
    input:
        month_NAV：該月所有淨值
        random_names：基金池的所有基金名
    output:
        整理出的features
        與features對齊的基金名
    '''
    @abstractmethod
    def getFeatures(self, NAV,names):
        pass
    
    '''
    input:回測年
    output:基金池（名字）
    '''
    @abstractmethod
    def poolDecide(self, year):
        pass
    
    def getNAV(self,names,start,end):
        start = self.getTimeStamp(start)
        end = self.getTimeStamp(end)
        data = pd.read_sql(sql='select * from price where date between ? and ? order by date asc',
                                 con=engine,index_col='date', params=[start,end])
        date = pd.read_sql(sql='select distinct date from price where date between ? and ? order by date asc',
                                 con=engine,index_col='date', params=[start,end]).index
        NAV = np.zeros((len(names),len(date)))
        for j in range (len(names)):
            temp = data[data['id'] == names[j]]
            NAV[j][0] = temp.iloc[0]['NAV']
            for i,day in enumerate(date[1:]):
                try:
                    NAV[j][i+1] = temp.loc[day]['NAV']
                except:
                    NAV[j][i+1] = NAV[j][i]
        return NAV
    
    def getTimeStamp(self,date):
        return int(pd.read_sql(sql='select strftime("%s",?)', con=engine, params=[date]).loc[0][0])
    def __init__(self,clusterStrategy):
        self.clusterStrategy = clusterStrategy
    
    def getWin(self):
        year = '2008'
        past_year = str(int(year)-1)
        names = self.poolDecide(past_year)
        NAV = self.getNAV(names,past_year+'-12-01',past_year+'-12-31')
        features,names,dissimilarity = self.getFeatures(NAV,names)
        clustering = ClusterMethod(self.clusterStrategy).startClustering(features)
        
        camp = pd.DataFrame(data=clustering.labels_, index=names,columns=['label'])
        choose_name = []
        for i in range(4):
            choose_name.append(camp[camp['label'] == i].sample(n=1).index[0])
            
        profit_choose = 0
        for name in choose_name:
            start = self.getTimeStamp(year+'-01-01')
            end = self.getTimeStamp(year+'-12-31')
            first = pd.read_sql(sql='select NAV from price where date > ? and id = ? limit 1'
                                ,con = engine, params=[start,name])['NAV'].values[0]
            last = pd.read_sql(sql='select NAV from price where date < ? and id = ? order by date desc limit 1'
                                ,con = engine, params=[end,name])['NAV'].values[0]
            interest = pd.read_sql(sql='select interest from interest where date between ? and ? and id = ?'
                                ,con = engine, params=[start,end,name])['interest'].sum()
            profit_choose += (last-first+interest)/first
        return profit_choose/4

In [7]:
'''
每日漲跌幅相關係數
'''
class UpDownRateSimilarity(CountWinRate):
        
    def getFeatures(self, NAV, names):
        length = len(NAV[0])-1
        rate = np.zeros((len(names),length))
        for j in range (len(names)):
            for i in range (length):
                rate[j][i] = (NAV[j][i+1] - NAV[j][i]) / NAV[j][i]
                
        temp = []
        for i,j in enumerate(rate):                        
            if np.cov(j) == 0:
                temp.append(i)                
        rate = np.delete(rate,temp,0)
        names = np.delete(names,temp,0)
        NAV = np.delete(NAV,temp,0)
        
        similarity = np.zeros((len(rate),len(rate)))
        for i in range(len(rate)):
            for j in range(len(rate)):
                corr = np.corrcoef(rate[i],rate[j])[0][-1]
                if i == j:
                    corr = 1
                similarity[i][j] = 1-(corr*0.5+0.5)       
#         features = []
#         for i in range (len(NAV)): 
#             features.append(np.append(rate[i], np.std(NAV[i], ddof=1)))
        return similarity,names,'precomputed'
    
    def poolDecide(self,year):
        start = self.getTimeStamp(year + '-12-01')
        end = self.getTimeStamp(year + '-12-31')
        names = pd.read_sql(sql='select distinct id from price where date between ? and ?',
                                 con=engine, params=[start,end])
        names = names['id'].sample(n=300).values
        return names

In [8]:
winRate = UpDownRateSimilarity(K_Means())

start = winRate.getTimeStamp('2016-01-01')
end = winRate.getTimeStamp('2016-12-31')
first = pd.read_sql(sql='select NAV from price where date > ? and id = "0050 元大台灣50" limit 1'
                    ,con = engine, params=[start])['NAV'].values[0]
last = pd.read_sql(sql='select NAV from price where date < ? and id = "0050 元大台灣50" order by date desc limit 1'
                   ,con = engine, params=[end])['NAV'].values[0]
interest = pd.read_sql(sql='select interest from interest where date between ? and ? and id = "0050 元大台灣50"'
                       ,con = engine, params=[start,end])['interest'].sum()
profit_0050 = (last-first+interest)/first

# count = 0
# for i in range(1000):
#     if (winRate.getWin()>profit_0050):
#         count +=1
#     print('目前是第' + str(i) + '次')
# print(count)

In [9]:
def multiCount(i):
    print('目前是第' + str(i) + '次')
    if (winRate.getWin()>profit_0050):
        return True
    return False

In [10]:
pool = mp.Pool()
ans = pool.map(multiCount, range(1000))

目前是第64次
目前是第32次
目前是第0次
目前是第128次
目前是第192次
目前是第96次
目前是第160次
目前是第224次
目前是第225次
目前是第193次
目前是第65次
目前是第161次
目前是第1次
目前是第97次
目前是第33次
目前是第129次
目前是第226次
目前是第194次
目前是第98次
目前是第162次
目前是第66次
目前是第34次
目前是第130次
目前是第2次
目前是第227次
目前是第99次
目前是第195次
目前是第163次
目前是第67次
目前是第131次
目前是第35次
目前是第3次
目前是第196次
目前是第100次
目前是第228次
目前是第164次
目前是第68次
目前是第132次
目前是第4次
目前是第36次
目前是第197次
目前是第101次
目前是第165次
目前是第229次
目前是第133次
目前是第69次
目前是第37次
目前是第5次
目前是第102次
目前是第198次
目前是第166次
目前是第38次
目前是第70次
目前是第134次
目前是第230次
目前是第6次
目前是第103次
目前是第199次
目前是第39次
目前是第135次
目前是第167次
目前是第7次
目前是第71次
目前是第231次
目前是第104次
目前是第40次
目前是第200次
目前是第168次
目前是第232次
目前是第136次
目前是第8次
目前是第72次
目前是第105次
目前是第41次
目前是第169次
目前是第201次
目前是第137次
目前是第9次
目前是第73次
目前是第233次
目前是第106次
目前是第42次
目前是第138次
目前是第170次
目前是第74次
目前是第202次
目前是第10次
目前是第234次
目前是第107次
目前是第171次
目前是第203次
目前是第139次
目前是第43次
目前是第75次
目前是第11次
目前是第235次
目前是第108次
目前是第204次
目前是第172次
目前是第140次
目前是第76次
目前是第12次
目前是第44次
目前是第236次
目前是第109次
目前是第205次
目前是第173次
目前是第141次
目前是第77次
目前是第13次
目前是第45次
目前是第237次
目前是第206次
目前是第110次
目前是第78次
目前是第174次
目前是第142次
目前是第

目前是第948次
目前是第853次
目前是第789次
目前是第885次
目前是第917次
目前是第822次
目前是第949次
目前是第981次
目前是第854次
目前是第790次
目前是第886次
目前是第823次
目前是第918次
目前是第950次
目前是第982次
目前是第855次
目前是第791次
目前是第887次
目前是第824次
目前是第919次
目前是第983次
目前是第951次
目前是第856次
目前是第792次
目前是第888次
目前是第920次
目前是第825次
目前是第984次
目前是第952次
目前是第857次
目前是第793次
目前是第889次
目前是第921次
目前是第826次
目前是第985次
目前是第953次
目前是第858次
目前是第794次
目前是第922次
目前是第890次
目前是第827次
目前是第986次
目前是第954次
目前是第859次
目前是第795次
目前是第923次
目前是第891次
目前是第828次
目前是第987次
目前是第955次
目前是第860次
目前是第796次
目前是第924次
目前是第892次
目前是第829次
目前是第988次
目前是第861次
目前是第956次
目前是第797次
目前是第925次
目前是第830次
目前是第893次
目前是第989次
目前是第862次
目前是第957次
目前是第798次
目前是第926次
目前是第894次
目前是第831次
目前是第990次
目前是第863次
目前是第799次
目前是第958次
目前是第895次
目前是第927次
目前是第991次
目前是第959次


In [11]:
count = 0
for i in ans:
    if (i):
        count +=1
print (count)

0
